In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print(f"TensorFlow version = {tf.__version__}\n")

SEED = 1350
np.random.seed(SEED)
tf.random.set_seed(SEED)

GESTURES = [
    "running",
    "walking",
    'idle'
]

SAMPLES_PER_GESTURE = 125
NUM_GESTURES = len(GESTURES)

inputs = []
outputs = []

for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(f"Processing index {gesture_index} for gesture '{gesture}'.")


  # got rid of the delimiter as the file is not tab separated "delimiter='\t'"
  df = pd.read_csv("/content/sample_data/" + gesture + ".csv", names = ['aX', 'aY', 'aZ'])
  print(f"Gesture '{gesture}' has {df.shape[0]} total samples")

  window_size = SAMPLES_PER_GESTURE
  stride = 80 # step size between windows

  num_windows = ((df.shape[0] - window_size) // stride) + 1


  # calculate the number of gesture recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)

  # print(f"\tThere are {num_recordings} recordings of the {gesture} gesture.")

  print(f"\tCreating {num_windows} windows for the {gesture} gesture.")

  for i in range(num_windows):
    start_idx = i * stride
    end_idx = start_idx + window_size

    # Make sure we don't go beyond the available data
    if end_idx <= df.shape[0]:
      X = df['aX'][start_idx:end_idx].values
      Y = df['aY'][start_idx:end_idx].values
      Z = df['aZ'][start_idx:end_idx].values
      tensor = np.vstack((X, Y, Z)).transpose()

      inputs.append(tensor)
      outputs.append(gesture_index)

# convert the list to numpy array
inputs = np.array(inputs)

mean = np.mean(inputs, axis=0)
std = np.std(inputs, axis=0)
inputs = (inputs - mean) / std

print(f"{np.min(inputs)=} {np.max(inputs)=}")

outputs = np.array(outputs)

# print("Data set parsing and preparation complete.")
print(inputs.shape, outputs.shape)

num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

print(f"Total samples before split: {num_inputs}")


# Split the recordings (group of samples) into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.", f"{inputs_train.shape=}, {outputs_train.shape=}")

print(np.unique(outputs_train))  # Should contain multiple classes
print(np.unique(outputs_test))   # Should contain multiple classes

TensorFlow version = 2.18.0

Processing index 0 for gesture 'running'.
Gesture 'running' has 4355 total samples
	Creating 53 windows for the running gesture.
Processing index 1 for gesture 'walking'.
Gesture 'walking' has 3756 total samples
	Creating 46 windows for the walking gesture.
Processing index 2 for gesture 'idle'.
Gesture 'idle' has 3550 total samples
	Creating 43 windows for the idle gesture.
np.min(inputs)=np.float64(-7.569344763264801) np.max(inputs)=np.float64(5.510077648639506)
(142, 125, 3) (142,)
Total samples before split: 142
Data set randomization and splitting complete. inputs_train.shape=(85, 125, 3), outputs_train.shape=(85,)
[0 1 2]
[0 1 2]


In [ ]:
# build the model and train it
model = tf.keras.Sequential([
    tf.keras.Input(shape=inputs_train.shape[1:], name='input'),
    # tf.keras.layers.Conv1D(8, 4, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=1e-4)),
    # tf.keras.layers.Dropout(0.1),
    tf.keras.layers.GlobalMaxPool1D(data_format='channels_last', name='global_max_pooling1d'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(NUM_GESTURES, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ global_max_pooling1d                 │ (None, 3)                   │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 227 (908.00 B)

 Trainable params: 227 (908.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    x=inputs_train,
    y=outputs_train,
    validation_data=[inputs_validate, outputs_validate],
    batch_size=32,
    shuffle=True,
    epochs=25
)

Epoch 1/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 278ms/step - accuracy: 0.4461 - loss: 0.8982 - val_accuracy: 0.3793 - val_loss: 0.9429
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.4461 - loss: 0.8420 - val_accuracy: 0.3793 - val_loss: 0.9100
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4872 - loss: 0.8030 - val_accuracy: 0.3793 - val_loss: 0.8801
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5342 - loss: 0.7710 - val_accuracy: 0.4483 - val_loss: 0.8546
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.5225 - loss: 0.7431 - val_accuracy: 0.4483 - val_loss: 0.8325
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.6125 - loss: 0.7171 - val_accuracy: 0.5172 - val_loss: 0.8123
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.6282 - loss: 0.6916 - val_accuracy: 0.6207 - val_loss: 0.7935
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.6751 - loss: 0.6662 - val_accuracy: 0.6207 - val_loss: 0

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)

# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model_quantized.tflite", "wb").write(tflite_model)

Saved artifact at '/tmp/tmp4aoct2bh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 125, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  134264332588560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134264332588176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134264332590096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134264332589712: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved artifact at '/tmp/tmp5mm9txxt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 125, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  134264332588560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134264332588176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134264332590096: TensorSpec(shape=

2912

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model_quantized.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 17,992 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
